### First, import the necessary libraries

In [1]:
from sklearn.cluster import KMeans
import numpy as np
import seaborn as sns
import csv
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout

In [2]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [3]:
from sklearn.preprocessing import MinMaxScaler

X_train = pd.read_csv('UNSW-NB15/a part of training and testing set/UNSW_NB15_testing-set.csv', header=0) # they flipped the names.
X_test = pd.read_csv('UNSW-NB15/a part of training and testing set/UNSW_NB15_training-set.csv', header=0)

X_train = X_train.dropna()
X_test=  X_test.dropna()

Separate labels from data

In [4]:
y_train = X_train.iloc[:,-1]
X_train = X_train.iloc[:,:-14]

y_test = X_test.iloc[:,-1]
X_test = X_test.iloc[:,:-14]

In [5]:
# categorical
for col in ['proto', 'service', 'state']:
    X_train[col] = pd.Categorical(X_train[col]).codes
    X_test[col] = pd.Categorical(X_test[col]).codes

X_train.pop('id')
X_test.pop('id')

0            1
1            2
2            3
3            4
4            5
         ...  
82327    82328
82328    82329
82329    82330
82330    82331
82331    82332
Name: id, Length: 82332, dtype: int64

In [6]:
# Remove columns created with "algorithms in C#" because they never say how they got them... not repeatable


## Scale the data (BIG, BIG PERFORMANCE INCREASE!!!)

In [7]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## what if I just threw a NN at it, or "How I became a data scientist in one easy step"

In [8]:
# define the keras model
batch_size = len(X_train)

model = tf.keras.Sequential()
model.add(Dense(64, input_dim=30, activation='relu'))
#model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))

# model.add(Dense(256, activation='relu'))
# #model.add(Dropout(0.2))

# # model.add(Dense(512, activation='relu'))
# # #model.add(Dropout(0.2))

# # # model.add(Dense(1024, activation='relu'))
# # # model.add(Dense(1024, activation='relu'))
# # # model.add(Dense(1024, activation='relu'))
# # # model.add(Dense(1024, activation='relu'))
# # # model.add(Dense(1024, activation='relu'))
# # # model.add(Dense(1024, activation='relu'))
# # # model.add(Dense(1024, activation='relu'))
# # # model.add(Dense(1024, activation='relu'))

# # model.add(Dense(512, activation='relu'))
# # #model.add(Dropout(0.2))

# model.add(Dense(256, activation='relu'))
#model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

In [9]:
from tensorflow.keras.optimizers import Adam
optim = Adam(learning_rate=0.001)

redlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=25, min_lr=0.00001, verbose=True)
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True, verbose=True)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = optim, metrics=['accuracy'])

In [10]:
#me shuffle train myself cuz tf doesnt do that when i use val split

shuffler = np.random.permutation(len(X_train))
X_train = X_train[shuffler]
y_train = y_train[shuffler]

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test.values))
test_dataset = test_dataset.shuffle(len(X_test)).batch(batch_size)

In [11]:
history = model.fit(X_train, y_train.values, epochs=2000, batch_size=batch_size, validation_split=0.20, callbacks=[redlr, earlystop] )

Epoch 1/2000
1/1 [==============================] - 1s 1s/step - loss: 0.6906 - accuracy: 0.6003 - val_loss: 0.6693 - val_accuracy: 0.7118
Epoch 2/2000
1/1 [==============================] - 0s 27ms/step - loss: 0.6690 - accuracy: 0.7157 - val_loss: 0.6530 - val_accuracy: 0.6885
Epoch 3/2000
1/1 [==============================] - 0s 27ms/step - loss: 0.6525 - accuracy: 0.6928 - val_loss: 0.6371 - val_accuracy: 0.6768
Epoch 4/2000
1/1 [==============================] - 0s 30ms/step - loss: 0.6364 - accuracy: 0.6818 - val_loss: 0.6206 - val_accuracy: 0.6766
Epoch 5/2000
1/1 [==============================] - 0s 27ms/step - loss: 0.6197 - accuracy: 0.6816 - val_loss: 0.6031 - val_accuracy: 0.6766
Epoch 6/2000
1/1 [==============================] - 0s 28ms/step - loss: 0.6020 - accuracy: 0.6816 - val_loss: 0.5847 - val_accura

In [12]:
model.save('less_features_fanning_layers_1024max_higher_starting_lr_no_dropout.h5')

In [13]:
y_pred = model.predict(X_test)

In [14]:
precision = tf.keras.metrics.Precision()
precision.update_state(y_test, y_pred)
precision.result().numpy()

0.72460824

In [15]:
recall = tf.keras.metrics.Recall()
recall.update_state(y_test, y_pred)
recall.result().numpy()

0.99966913

In [16]:
accuracy = tf.keras.metrics.BinaryAccuracy()
accuracy.update_state(y_test, y_pred)
accuracy.result().numpy()

0.7906282

In [17]:
actual_prediction = np.round(y_pred)

# actually normal, predicted normal
normal_traffic_indices = np.argwhere(y_test.to_numpy()==0)
normal_traffic_indices = normal_traffic_indices.reshape((len(normal_traffic_indices),))
attack_traffic_indices = np.argwhere(y_test.to_numpy()==1)
attack_traffic_indices = attack_traffic_indices.reshape((len(attack_traffic_indices),))

In [18]:
tn = len(np.where(np.take(actual_prediction, normal_traffic_indices, axis=0)==0)[0])
norm = len(normal_traffic_indices)
print(f'classified {tn} out of {norm} normal traffic patterns as normal {tn/norm}%')

classified 19777 out of 37000 normal traffic patterns as normal 0.5345135135135135%


In [19]:
fn = len(np.where(np.take(actual_prediction, attack_traffic_indices, axis=0)==0)[0])
attack = len(attack_traffic_indices)
print(f'classified {fn} out of {attack} attack traffic patterns as normal {fn/attack}%')

classified 15 out of 45332 attack traffic patterns as normal 0.00033089208506132534%


In [20]:
fp = len(np.where(np.take(actual_prediction, normal_traffic_indices, axis=0)==1)[0])
print(f'classified {fp} out of {norm} normal traffic patterns as attack {fp/norm}%')

classified 17223 out of 37000 normal traffic patterns as attack 0.4654864864864865%


In [21]:
tp = len(np.where(np.take(actual_prediction, attack_traffic_indices, axis=0)==1)[0])
print(f'classified {tp} out of {attack} attack traffic patterns as attack {tp/attack}%')

classified 45317 out of 45332 attack traffic patterns as attack 0.9996691079149387%


In [22]:
tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=False)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
